In [1]:
# python peripherals
import sys
from pathlib import Path
from typing import Callable, List
sys.path.append('../../.')

# numpy
import numpy

# torch
import torch

# matplotlib
import matplotlib
import matplotlib.pyplot
import matplotlib.axes

# deep-signature
from deep_signature.core.base import SeedableObject
from deep_signature.manifolds.planar_curves.implementation import PlanarCurvesManager, PlanarCurve
from deep_signature.manifolds.planar_curves.groups import EuclideanGroup, SimilarityGroup, EquiaffineGroup, AffineGroup
from deep_signature.core import discrete_distributions
from deep_signature.training import datasets
from deep_signature.training.networks import DeepSignaturesNet
from deep_signature.training.activations import Sine
from deep_signature.manifolds.planar_curves.evaluation import PlanarCurvesApproximatedSignatureComparator, PlanarCurvesShapeMatchingEvaluator
from deep_signature.manifolds.planar_curves.groups import EquiaffineGroup

# pandas
import pandas

# numba
import numba

SeedableObject.set_seed(seed=42)

C:\Users\Roy\miniconda3\envs\deep-signature\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# planar_curves_manager_smooth = PlanarCurvesManager(curves_file_paths=[Path("C:/deep-signature-data-new/curves/train/2022-12-31-15-12-51/curves.npy")])
planar_curves_manager_smooth = PlanarCurvesManager(curves_file_paths=[Path("C:/deep-signature-data/curves/train/curves.npy")])

In [3]:
# planar_curve = planar_curves_manager_smooth.get_random_planar_curve()
#
# fig, ax = matplotlib.pyplot.subplots(nrows=1, ncols=1, figsize=(5, 5))
# planar_curve.plot_scattered_curve(ax=ax, cmap=None, color='red')
#
# fig, axes = matplotlib.pyplot.subplots(nrows=3, ncols=1, figsize=(20, 10))
# planar_curve.plot_euclidean_signature(ax=axes, force_limits=True)
# matplotlib.pyplot.show()

In [8]:
signatures = []
points_count = []
curves_count = planar_curves_manager_smooth.planar_curves_count
for i in range(curves_count):
    curve = planar_curves_manager_smooth.planar_curves[i]
    signatures.append(curve.approximate_euclidean_signature())
    points_count.append(curve.points_count)

In [18]:
samples_count = [10, 100, 1000, 10000, 100000, 1000000, 10000000]

corrs = []
times = 5
for k in samples_count:
    current_corrs = numpy.zeros(shape=times)
    for j in range(times):
        samples = numpy.zeros(shape=[k, 2])
        for i in range(k):
            curve_index = numpy.random.randint(curves_count)
            point_index = numpy.random.randint(points_count[curve_index])
            signature = signatures[curve_index]
            samples[i] = signature[point_index]
        current_corrs[j] = numpy.abs(numpy.corrcoef([samples[:, 0], samples[:, 1]])[0, 1])

    corrs.append(numpy.mean(current_corrs))
    print(numpy.mean(current_corrs))


data = {
    'samples_count': samples_count,
    'corr': corrs,
}

df = pandas.DataFrame(data=data)
df.to_csv("C:/deep-signature-data-new/notebooks_output/pearson.csv")

0.7015034095971011
0.5792430530203232
0.4695659766688518
0.39196448714575743
0.181101181049931
0.09372122129405304
0.022948769081406965
